# Imports

In [67]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv


# Set Paths

In [68]:
# Define the project path and a list of 5 ABF file names
project_path = 'C:\\Users\\ikmor\\OneDrive\\Documents\\SNR AND OBR pilot study\\'
abf_files = [
    f'{project_path}PTX10 CNQX A3.abf',
    f'{project_path}PTX8R3 ACCIN.abf',
    f'{project_path}PTX7L2 ACCOM.abf',
    f'{project_path}PTX09R5 ACCOM3.abf',
    f'{project_path}PTX6L2 ACCOM.abf',
    f'{project_path}CNQX1L3 A3.abf'
    ]




In [69]:
# Initialize a 2D list to store action potential counts for each sweep across all files
sweep_ap_counts = [[] for _ in range(10)]  # 7 sweeps

LJP = 13  # Liquid Junction Potential in mV

# Loop through each ABF file
for abf_file in abf_files:
    # Load the ABF file
    abf = pyabf.ABF(abf_file)
    
    # Define sweep numbers
    sweep_numbers = range(10)  # 7 sweeps

    # Loop through each sweep
    for sweep in sweep_numbers:
        abf.setSweep(sweep, channel=0)
        
        # Define the time segment for analysis (in seconds)
        start_time = 2.5  # Start time in seconds
        end_time = 7.5    # End time in seconds

        # Extract voltage (sweepY) and time (sweepX)
        voltage = abf.sweepY - LJP  # Voltage in mV
        times = abf.sweepX    # Time in seconds

        # Find indices corresponding to the desired time segment
        start_index = np.argmin(np.abs(times - start_time))
        end_index = np.argmin(np.abs(times - end_time))

        # Extract the specific portion of the data for analysis
        segment_voltage = voltage[start_index:end_index]

        # Count the number of times the waveform crosses 0mV
        zero_crossings = np.sum(np.diff(np.sign(segment_voltage)) != 0)
        
        # Calculate the number of action potentials (half of zero crossings)
        ap_count = zero_crossings // 2

        # Store the AP count for this sweep in the corresponding list
        sweep_ap_counts[sweep].append(ap_count)

# Calculate average AP counts for each sweep
average_ap_counts = [np.mean(counts) for counts in sweep_ap_counts]

# Calculate standard error for each sweep
standard_errors = [np.std(counts, ddof=1) / np.sqrt(len(counts)) if len(counts) > 1 else 0 for counts in sweep_ap_counts]

# Output results to CSV
output_file = f'{project_path}average_ap_counts_with_errors.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Sweep', 'Average Action Potentials', 'Standard Error'])
    for sweep, (avg_count, std_err) in enumerate(zip(average_ap_counts, standard_errors)):
        writer.writerow([sweep + 1, avg_count, std_err])

print(f"Results with standard errors have been saved to {output_file}")

Results with standard errors have been saved to C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\average_ap_counts_with_errors.csv
